In [2]:
# Generate notebook download link
from IPython.display import FileLink
print('To download this notebook, right click on the link and Save link as...')
FileLink('tutorial_merge.ipynb')

To download this notebook, right click on the link and Save link as...


X:\btcache\scratch\johnsonc\jupyter\Tutorial\Tutorial_GitLab\tutorial_merge.ipynb

In [3]:
import riptable as rt
import numpy as np

In [4]:
# Display all Dataset columns -- the default max is 9.
rt.Display.options.COL_ALL = True

# Render up to 100MM before showing in scientific notation.
rt.Display.options.E_MAX = 100_000_000

# Truncate small decimals, rather than showing infinitesimal scientific notation.
rt.Display.options.P_THRESHOLD = 0

# Put commas in numbers.
rt.Display.options.NUMBER_SEPARATOR = True

# Turn on Riptable autocomplete (start typing, then press Tab to see options).
rt.autocomplete()

# Merge Datasets

Merging gives you more flexibility to bring data from different Datasets together. 

A merge operation connects rows in Datasets using a "key" column that the Datasets have in common.

Riptable's two main Dataset merge functions are `merge_lookup()` and `merge_asof()`. Generally speaking, `merge_lookup()` aligns data based on identical keys, while `merge_asof()` aligns data based on the nearest key.

For more general merges, `merge2()` does datbase-style left, right, inner, and outer joins.

## `merge_lookup()`

Let's start with `merge_lookup()`. It's common to have one Dataset that has most of the information you need, and another, usually smaller Dataset that has information you want to add to the first Dataset to enrich it.

Here we'll create a larger Dataset with symbols and size values, and a smaller Dataset that has symbols associated with trader names. We'll use the shared Symbol column as the key to add the trader info to the larger Dataset.

In [5]:
rng = np.random.default_rng(seed=42)

N = 50

# Larger Dataset
ds = rt.Dataset({'Symbol': rng.choice(['GME', 'AMZN', 'TSLA', 'SPY'], N),
                 'Size': rng.integers(1, 1000, N),})

# Smaller Dataset, with data used to enrich the larger Dataset
ds_symbol_trader = rt.Dataset({'Symbol': ['GME', 'TSLA', 'SPY', 'AMZN'],
                           'Trader': ['Nate', 'Elon', 'Josh', 'Dan']})

In [6]:
ds.head()

#,Symbol,Size
0,GME,760
1,SPY,195
2,TSLA,364
3,AMZN,467
4,AMZN,498
5,SPY,44
6,GME,547
7,TSLA,155
8,GME,743
9,GME,683


In [7]:
ds_symbol_trader

#,Symbol,Trader
0,GME,Nate
1,TSLA,Elon
2,SPY,Josh
3,AMZN,Dan


### `merge_lookup()` with Key Columns That Have the Same Name

Now we'll use `merge_lookup()` to add the trader information to the larger Dataset. `merge_lookup()` will align the data based on exact matches in the shared Symbol column.

A note about terms: When you merge two Datasets, the Dataset you're merging data into is the *left Dataset*; the one you're getting data from is the *right Dataset*. 

Here, we call `merge_lookup()` on our left Dataset, `ds`. We pass it the name of the right Dataset, and tell it what column to use as the key.

In [8]:
ds.merge_lookup(ds_symbol_trader, on='Symbol')

#,Symbol,Size,Trader
0,GME,760,Nate
1,SPY,195,Josh
2,TSLA,364,Elon
3,AMZN,467,Dan
4,AMZN,498,Dan
5,SPY,44,Josh
6,GME,547,Nate
7,TSLA,155,Elon
8,GME,743,Nate
9,GME,683,Nate


The left Dataset now has the trader information, correctly aligned.

You can also use the following syntax, passing `merge_lookup()` the names of the left and right Datasets, along with the key:

In [9]:
rt.merge_lookup(ds, ds_symbol_trader, on='Symbol')

#,Symbol,Size,Trader
0,GME,760,Nate
1,SPY,195,Josh
2,TSLA,364,Elon
3,AMZN,467,Dan
4,AMZN,498,Dan
5,SPY,44,Josh
6,GME,547,Nate
7,TSLA,155,Elon
8,GME,743,Nate
9,GME,683,Nate


### `merge_lookup` with Key Columns That Have Different Names

If the key column has a different name in each Dataset, just specify each column name with `left_on` and `right_on`.

In [10]:
# For illustrative purposes, rename the key column in the right Dataset.
ds_symbol_trader.col_rename('Symbol', 'UnderlyingSymbol')

ds.merge_lookup(ds_symbol_trader, left_on='Symbol', right_on='UnderlyingSymbol')

#,Symbol,Size,UnderlyingSymbol,Trader
0,GME,760,GME,Nate
1,SPY,195,SPY,Josh
2,TSLA,364,TSLA,Elon
3,AMZN,467,AMZN,Dan
4,AMZN,498,AMZN,Dan
5,SPY,44,SPY,Josh
6,GME,547,GME,Nate
7,TSLA,155,TSLA,Elon
8,GME,743,GME,Nate
9,GME,683,GME,Nate


Notice that when the key columns have different names, both are kept. If you want keep only certain columns from the left or right Dataset, you can specify them with `columns_left` or `columns_right`.

In [11]:
ds.merge_lookup(ds_symbol_trader, left_on='Symbol', right_on='UnderlyingSymbol', 
                columns_right='Trader')

#,Symbol,Size,Trader
0,GME,760,Nate
1,SPY,195,Josh
2,TSLA,364,Elon
3,AMZN,467,Dan
4,AMZN,498,Dan
5,SPY,44,Josh
6,GME,547,Nate
7,TSLA,155,Elon
8,GME,743,Nate
9,GME,683,Nate


### Note: `merge_lookup()` Keeps Only the Keys in the Left Dataset

One thing to note about `merge_lookup()` is that it keeps only the keys are that are in the left Dataset (it's equivalent to a SQL left join). If there are keys in the right Dataset that aren't in the left Dataset, they're discarded in the merged data.

In [12]:
# Create a right Dataset with an extra symbol key ('MSFT').
ds_symbol_trader2 = rt.Dataset({'Symbol': ['GME', 'TSLA', 'SPY', 'AMZN', 'MSFT'], 
                           'Trader': ['Nate', 'Elon', 'Josh', 'Dan', 'Lauren']})
                           
# Change 'UnderlyingSymbol' back to 'Symbol' for simplicity.
ds_symbol_trader.col_rename('UnderlyingSymbol', 'Symbol')

ds.merge_lookup(ds_symbol_trader2, on='Symbol', columns_right='Trader')._A

#,Symbol,Size,Trader
0,GME,760,Nate
1,SPY,195,Josh
2,TSLA,364,Elon
3,AMZN,467,Dan
4,AMZN,498,Dan
5,SPY,44,Josh
6,GME,547,Nate
7,TSLA,155,Elon
8,GME,743,Nate
9,GME,683,Nate


The 'MSFT' key, which appears only in the right Dataset, is ignored.

### `merge_lookup()` with Overlapping Columns That Aren't Keys

As we saw above, if the two key columns have the same name in both Datasets, only one is kept. For columns that aren't used as keys, you'll get a name collision error when you try to merge. 

In [13]:
try:
    rt.merge_lookup(ds, ds_symbol_trader, on='Symbol')
except ValueError as e:
    print("ValueError:", e)

If you want to keep both columns, add a suffix to each column name to disambiguate them.

In [14]:
# Add a Size column to the right Dataset
ds_symbol_trader.Size = rng.integers(1, 1000, 4)

rt.merge_lookup(ds, ds_symbol_trader, on='Symbol', suffixes=('_1', '_2'))

#,Symbol,Size_1,Trader,Size_2
0,GME,760,Nate,832
1,SPY,195,Josh,805
2,TSLA,364,Elon,200
3,AMZN,467,Dan,8
4,AMZN,498,Dan,8
5,SPY,44,Josh,805
6,GME,547,Nate,832
7,TSLA,155,Elon,200
8,GME,743,Nate,832
9,GME,683,Nate,832


### `merge_lookup()` with a Right Dataset That Has Duplicate Keys

If the right Dataset has more than one match for a unique key in the left Dataset, you can specify whether to use the first or the last match encountered in the right Dataset.

In [15]:
# Create a right Dataset with a second GME key, associated to Lauren
ds_symbol_trader3 = rt.Dataset({'Symbol': ['GME', 'TSLA', 'SPY', 'AMZN', 'GME'], 
                           'Trader': ['Nate', 'Elon', 'Josh', 'Dan', 'Lauren']})

ds_symbol_trader3

#,Symbol,Trader
0,GME,Nate
1,TSLA,Elon
2,SPY,Josh
3,AMZN,Dan
4,GME,Lauren


We'll keep the last match.

In [16]:
ds.merge_lookup(ds_symbol_trader3, on='Symbol', columns_right='Trader', keep='last')

#,Symbol,Size,Trader
0,GME,760,Lauren
1,SPY,195,Josh
2,TSLA,364,Elon
3,AMZN,467,Dan
4,AMZN,498,Dan
5,SPY,44,Josh
6,GME,547,Lauren
7,TSLA,155,Elon
8,GME,743,Lauren
9,GME,683,Lauren


## `merge_asof()`

In a `merge_asof()`, Riptable matches on the nearest key rather than an equal key. 

This is useful for merges based on keys that are times, where the times in one Dataset are not an exact match for the times in another Dataset, but they're close enough to be used to merge the data. 

Note: To most efficiently find the nearest match, `merge_asof()` requires both key columns to be sorted. The key columns must also be numeric, such as a datetime, integer, or float. You can check whether a column is sorted with `issorted()`, or just sort it using `sort_inplace()`. (If the key columns aren't sorted, Riptable will give you an error when you try to merge.)

With `merge_asof()`, you need to specify how you want to find the closest match:
- `direction='forward'` matches based on the closest key in the right Dataset that's greater than the key in the left Dataset.
- `direction='backward'` matches based on the closest key in the right Dataset that's less than the key in the left Dataset.
- `direction='nearest'` matches based on the closest key in the right Dataset, regardless of whether it's greater than or less than the key in the left Dataset.

Let's see an example based on closest times. The left Dataset has three trades and their times. The right Dataset has spot prices and times that are not all exact matches. We'll merge the spot prices from the right Dataset by getting the values associated with the nearest earlier times.

In [17]:
# Left Dataset with trades and times
ds = rt.Dataset({'Symbol': ['AAPL', 'AMZN', 'AAPL'], 
                 'Venue': ['A', 'I', 'A'],
                 'Time': rt.TimeSpan(['09:30', '10:00', '10:20'])})

# Right Dataset with spot prices and nearby times
spot_ds = rt.Dataset({'Symbol': ['AMZN', 'AMZN', 'AMZN', 'AAPL', 'AAPL', 'AAPL'],
                      'Spot Price': [2000.0, 2025.0, 2030.0, 500.0, 510.0, 520.0],
                      'Time': rt.TimeSpan(['09:30', '10:00', '10:25', '09:25', '10:00', '10:21'])})

In [18]:
ds

#,Symbol,Venue,Time
0,AAPL,A,09:30:00.000000000
1,AMZN,I,10:00:00.000000000
2,AAPL,A,10:20:00.000000000


In [19]:
spot_ds

#,Symbol,Spot Price,Time
0,AMZN,"2,000.00",09:30:00.000000000
1,AMZN,"2,025.00",10:00:00.000000000
2,AMZN,"2,030.00",10:25:00.000000000
3,AAPL,500.00,09:25:00.000000000
4,AAPL,510.00,10:00:00.000000000
5,AAPL,520.00,10:21:00.000000000


But first, we need to make sure both Datasets are sorted by the key columns -- in this case, the Time columns. It looks like `ds` has sorted times, but `spot_ds` doesn't.

In [20]:
spot_ds.sort_inplace('Time')

#,Symbol,Spot Price,Time
0,AAPL,500.00,09:25:00.000000000
1,AMZN,"2,000.00",09:30:00.000000000
2,AMZN,"2,025.00",10:00:00.000000000
3,AAPL,510.00,10:00:00.000000000
4,AAPL,520.00,10:21:00.000000000
5,AMZN,"2,030.00",10:25:00.000000000


Now we can merge based on the nearest earlier time. But not just any nearest earlier time -- we want to make sure it's the nearest earlier time associated with the same symbol. We use the optional `by` parameter to make sure we match on the symbol before getting the nearest earlier time.

In [21]:
ds.merge_asof(spot_ds, on='Time', by='Symbol', direction='backward')

#,Time,Symbol,Venue,Spot Price
0,09:30:00.000000000,AAPL,A,500.00
1,10:00:00.000000000,AMZN,I,"2,025.00"
2,10:20:00.000000000,AAPL,A,510.00


We can see that both AAPL trades were matched based on the nearest earlier time.

Merge based on the nearest later time:

In [22]:
ds.merge_asof(spot_ds, on='Time', by='Symbol', direction='forward')

#,Time,Symbol,Venue,Spot Price
0,09:30:00.000000000,AAPL,A,510.00
1,10:00:00.000000000,AMZN,I,"2,025.00"
2,10:20:00.000000000,AAPL,A,520.00


Both AAPL trades were matched based on the nearest later time.

Here, we get the spot price associated with whatever time is nearest, whether it's earlier or later.

In [23]:
ds.merge_asof(spot_ds, on='Time', by='Symbol', direction='nearest')

#,Time,Symbol,Venue,Spot Price
0,09:30:00.000000000,AAPL,A,500.00
1,10:00:00.000000000,AMZN,I,"2,025.00"
2,10:20:00.000000000,AAPL,A,520.00


For the first AAPL trade, the nearest time is earlier. For the second AAPL trade, the nearest time is later.


We won't spend time on examples of `merge2()`, which is Riptable's more general merge function that does database-style joins (left, right, inner, outer). Check out the API Reference for details.

Next, we'll briefly cover a couple of ways to change the shape of your Dataset: [Reshape Data with Pivot and Transpose](tutorial_reshape.ipynb).

<br>
<br>

---

Questions or comments about this guide? Email RiptableDocumentation@sig.com.